In [1]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [2]:
train_path = '../data_to_model/keras_data/model_df_full_narrative.csv'
train_df = pd.read_csv(train_path)
train_df

,cpsc_case_number,narrative,primary_mechanism
0,100145411,67 yof pt fell on the floor playing pickle bal...,"Falls, trips, etc."
1,100417928,63yof playing pickleball in the hot sun and pa...,Heat stroke
2,100417973,66yom strained lower leg playing pickle ball,Other mechanism
3,100417997,60yof pt playing pickle ball and fell sustaine...,"Falls, trips, etc."
4,100454409,70 yom pt injured knee while playing pickle ba...,Other mechanism
...,...,...,...
1986,200101825,68 yof tripped and fell while playing tennis. ...,"Falls, trips, etc."
1987,200102934,76yof was hit in the eye with a tennis ball dx...,Hit with various obj
1988,200110467,61yof was playing tennis when she developed pa...,Undetermined/unknown
1989,200124689,70yof fell while playing tennis dx: strained u...,"Falls, trips, etc."


In [5]:
text_np = np.array(train_df['narrative'])
train_labels_np = np.array(train_df['primary_mechanism'])
train_labels_np_bin = np.where(train_labels_np == 'Multi-sport', 'Multi-sport', 'Other')
print(train_labels_np[0:30])
print(train_labels_np_bin[0:30])

['Falls, trips, etc.' 'Heat stroke' 'Other mechanism' 'Falls, trips, etc.'
 'Other mechanism' 'Falls, trips, etc.' 'Falls, trips, etc.'
 'Other mechanism' 'Other mechanism' 'Falls, trips, etc.'
 'Hit with various obj' 'Falls, trips, etc.' 'Falls, trips, etc.'
 'cardio/sync' 'Falls, trips, etc.' 'Other mechanism' 'Other mechanism'
 'Other mechanism' 'Other mechanism' 'Falls, trips, etc.'
 'Falls, trips, etc.' 'Hit with various obj' 'Other mechanism'
 'Multi-sport' 'Falls, trips, etc.' 'Other mechanism' 'Falls, trips, etc.'
 'Falls, trips, etc.' 'Falls, trips, etc.' 'Falls, trips, etc.']
['Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Multi-sport' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other']


In [4]:
#l_list = []
#for txt in text_np:
#    l_list.append( len(txt) )
    
#l_list[0:10]

#max(l_list)

In [5]:
maxlen = 207
training_samples = 1491
validation_samples = 500
max_words = 100000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_np)
sequences = tokenizer.texts_to_sequences(text_np)


In [6]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', train_labels_np.shape)


Found 1857 unique tokens.
Shape of data tensor: (1991, 207)
Shape of label tensor: (1991,)


In [7]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
train_labels_np = train_labels_np[indices]
x_train = data[:training_samples]
y_train = train_labels_np[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = train_labels_np[training_samples: training_samples + validation_samples]

In [8]:
label_encoder = LabelEncoder()

y_train_integer_encoded = label_encoder.fit_transform(y_train)
y_val_integer_encoded = label_encoder.fit_transform(y_val)

#y_train_integer_encoded
#y_train_integer_encoded
y_train_one_hot = to_categorical(y_train_integer_encoded, num_classes=7)
y_val_one_hot = to_categorical(y_val_integer_encoded, num_classes=7)

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 207, 8)            800000    
_________________________________________________________________
flatten (Flatten)            (None, 1656)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 11599     
Total params: 811,599
Trainable params: 811,599
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(x_train, y_train_one_hot,
epochs=80,
batch_size=32,
validation_data=(x_val, y_val_one_hot))

Epoch 1/80
47/47 [==============================] - 1s 21ms/step - loss: 0.3382 - accuracy: 0.4755 - val_loss: 0.3182 - val_accuracy: 0.4840
Epoch 2/80
47/47 [==============================] - 1s 12ms/step - loss: 0.3172 - accuracy: 0.4842 - val_loss: 0.3019 - val_accuracy: 0.4840
Epoch 3/80
47/47 [==============================] - 1s 12ms/step - loss: 0.2963 - accuracy: 0.5017 - val_loss: 0.2800 - val_accuracy: 0.5040
Epoch 4/80
47/47 [==============================] - 1s 12ms/step - loss: 0.2716 - accuracy: 0.5473 - val_loss: 0.2584 - val_accuracy: 0.5480
Epoch 5/80
47/47 [==============================] - 1s 12ms/step - loss: 0.2473 - accuracy: 0.6117 - val_loss: 0.2370 - val_accuracy: 0.6840
Epoch 6/80
47/47 [==============================] - 1s 11ms/step - loss: 0.2259 - accuracy: 0.6720 - val_loss: 0.2181 - val_accuracy: 0.7100
Epoch 7/80
47/47 [==============================] - 1s 11ms/step - loss: 0.2075 - accuracy: 0.7009 - val_loss: 0.2016 - val_accuracy: 0.7080
Epoch 8/80
47

In [12]:
model.save_weights('../Model Objects/keras_outv1.h5')

In [13]:
def getModel():
    my_dim = 8
    
    model = Sequential()
    model.add(Embedding(max_words, my_dim, input_length=maxlen))
    model.add(Flatten())
#    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    return model

In [14]:
# Different configurations to try
epochs_list = [30, 60, 90]
batch_sizes = [16, 32, 64]
#embedding_dim = 8
# Dictionary to store the history of each configuration
history_dict = {}

for epochs in epochs_list:
    for batch_size in batch_sizes:
        # Get a fresh instance of the model for each run
        model = getModel()

        # Compile the model
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        history = model.fit(x_train, y_train_one_hot,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(x_val, y_val_one_hot),
                            verbose=2)  # Set verbose to 2 for less output

        # Save the history of this configuration
        history_dict[f"epochs_{epochs}_batch_{batch_size}"] = history.history

        # Optionally, you can print out the final validation accuracy for each configuration
        final_val_accuracy = history.history['val_accuracy'][-1]
        print(f"Epochs: {epochs}, Batch size: {batch_size}, Validation accuracy: {final_val_accuracy}")

# Now you can analyze the history_dict to see which configuration performed best


Epoch 1/30
94/94 - 1s - loss: 1.5095 - accuracy: 0.4695 - val_loss: 1.3897 - val_accuracy: 0.4840
Epoch 2/30
94/94 - 1s - loss: 1.3636 - accuracy: 0.4930 - val_loss: 1.2377 - val_accuracy: 0.5360
Epoch 3/30
94/94 - 1s - loss: 1.1646 - accuracy: 0.5902 - val_loss: 1.0387 - val_accuracy: 0.6500
Epoch 4/30
94/94 - 1s - loss: 0.9838 - accuracy: 0.6975 - val_loss: 0.8879 - val_accuracy: 0.7300
Epoch 5/30
94/94 - 1s - loss: 0.8485 - accuracy: 0.7371 - val_loss: 0.7874 - val_accuracy: 0.7680
Epoch 6/30
94/94 - 1s - loss: 0.7422 - accuracy: 0.7713 - val_loss: 0.7097 - val_accuracy: 0.7960
Epoch 7/30
94/94 - 1s - loss: 0.6610 - accuracy: 0.7934 - val_loss: 0.6557 - val_accuracy: 0.8060
Epoch 8/30
94/94 - 1s - loss: 0.5923 - accuracy: 0.8216 - val_loss: 0.6050 - val_accuracy: 0.8280
Epoch 9/30
94/94 - 1s - loss: 0.5363 - accuracy: 0.8364 - val_loss: 0.5848 - val_accuracy: 0.8300
Epoch 10/30
94/94 - 1s - loss: 0.4876 - accuracy: 0.8571 - val_loss: 0.5466 - val_accuracy: 0.8380
Epoch 11/30
94/94 -

In [15]:
history_dict

{'epochs_30_batch_16': {'loss': [1.509526252746582,
   1.3635553121566772,
   1.1646208763122559,
   0.9837902188301086,
   0.8485177755355835,
   0.742249071598053,
   0.6610434651374817,
   0.5922733545303345,
   0.536267876625061,
   0.48755964636802673,
   0.44664090871810913,
   0.4109458923339844,
   0.3785935342311859,
   0.3484634757041931,
   0.3227599859237671,
   0.29809239506721497,
   0.2772292196750641,
   0.2570899724960327,
   0.23897084593772888,
   0.22147691249847412,
   0.20561937987804413,
   0.1911049485206604,
   0.17673826217651367,
   0.16465657949447632,
   0.152627632021904,
   0.14159518480300903,
   0.13084761798381805,
   0.1198643296957016,
   0.1115197241306305,
   0.10371623933315277],
  'accuracy': [0.46948355436325073,
   0.49295774102211,
   0.5902079343795776,
   0.6975184679031372,
   0.737089216709137,
   0.7712944149971008,
   0.7934272289276123,
   0.8215962648391724,
   0.8363514542579651,
   0.8571428656578064,
   0.8692153096199036,
   0.8846